# Машинное обучение, ФКН ВШЭ

## Практическое задание 5. Решающие деревья

### Общая информация
Дата выдачи: 29.11.2023

Мягий дедлайн: 23:59 12.12.2023

Жестокий дедлайн: 23:59 14.12.2023

### О задании

Задание состоит из двух разделов:
1. В первом разделе вы научитесь применять деревья из sklearn для задачи классификации. Вы посмотрите какие разделяющие поверхности деревья строят для различных датасетов и проанализируете их зависимость от различных гиперпараметров.
2. Во втором разделе вы попробуете реализовать свое решающее дерево и сравните его со стандартное имплиментацией из sklearn. Вы также протестируете деревья на более сложных датасетах и сравните различные подходы к кодированию категориальных признаков.

### Оценивание и штрафы
Каждая из задач имеет определенную «стоимость» (указана в скобках около задачи). Максимально допустимая оценка за работу — 10 баллов.

Сдавать задание после указанного срока сдачи нельзя. При выставлении неполного балла за задание в связи с наличием ошибок на усмотрение проверяющего предусмотрена возможность исправить работу на указанных в ответном письме условиях.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов (подробнее о плагиате см. на странице курса). Если вы нашли решение какого-то из заданий (или его часть) в открытом источнике, необходимо указать ссылку на этот источник в отдельном блоке в конце вашей работы (скорее всего вы будете не единственным, кто это нашел, поэтому чтобы исключить подозрение в плагиате, необходима ссылка на источник).

Неэффективная реализация кода может негативно отразиться на оценке.


### Формат сдачи
Задания сдаются через систему anytask. Посылка должна содержать:
* Ноутбук homework-practice-05-trees-Username.ipynb
* Модуль hw5code.py
* Ссылки на посылки в Яндекс.Контесте для обеих задач

В контест https://contest.yandex.ru/contest/56884/problems/ нужно отправить файл hw5code.py с реализованными функциями и классами.

Username — ваша фамилия и имя на латинице именно в таком порядке

Для удобства проверки самостоятельно посчитайте свою максимальную оценку (исходя из набора решенных задач) и укажите ниже:

__Оценка:__

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from matplotlib.colors import Colormap, ListedColormap
import pandas as pd
from sklearn.model_selection import train_test_split
import seaborn as sns
sns.set(style='whitegrid')

RANDOM_STATE = 42

import warnings
warnings.filterwarnings('ignore')

# 1. Решающие деревья. Визуализация.

В этой части мы рассмотрим два простых двумерных датасета сделанных с помощью `make_moons`, `make_circles` и посмотрим как ведет себя разделяющая поверхность в зависимости от различных гиперпараметров.

In [ ]:
from sklearn.datasets import make_moons, make_circles, make_classification
datasets = [
    make_circles(noise=0.2, factor=0.5, random_state=42),
    make_moons(noise=0.2, random_state=42),
    make_classification(n_classes=3, n_clusters_per_class=1, n_features=2, class_sep=.8, random_state=3,
                        n_redundant=0., )
]

In [ ]:
palette = sns.color_palette(n_colors=3)
cmap = ListedColormap(palette)

In [ ]:
plt.figure(figsize=(15, 4))
for i, (x, y) in enumerate(datasets):
    plt.subplot(1, 3, i + 1)
    plt.scatter(x[:, 0], x[:, 1], c=y, cmap=cmap, alpha=.8)

__Задание 1. (1 балл)__

Для каждого датасета обучите решающее дерево с параметрами по умолчанию, предварительно разбив выборку на обучающую и тестовую. Постройте разделящие поверхности (для этого воспользуйтесь функцией `plot_surface`, пример ниже). Посчитайте accuracy на обучающей и тестовой выборках. Сильно ли деревья переобучились?

In [ ]:
def plot_surface(clf, X, y):
    plot_step = 0.01
    palette = sns.color_palette(n_colors=len(np.unique(y)))
    cmap = ListedColormap(palette)
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, plot_step),
                         np.arange(y_min, y_max, plot_step))
    plt.tight_layout(h_pad=0.5, w_pad=0.5, pad=2.5)

    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    cs = plt.contourf(xx, yy, Z, cmap=cmap, alpha=0.3)

    plt.scatter(X[:, 0], X[:, 1], c=y, cmap=cmap, alpha=.7,
                edgecolors=np.array(palette)[y], linewidths=2)

In [ ]:
# Пример:
from sklearn.linear_model import LinearRegression
X, y = datasets[2]
lr  = LinearRegression().fit(X, y)
plot_surface(lr, X, y)

In [ ]:
from sklearn.metrics import accuracy_score

dataset_name = ["moons", "circles", "classification"]

splitted_data = []

def train_trees(datasets, dataset: int = None, **model_kwargs: dict):
    if dataset is not None:
        datasets = [datasets[dataset]]
    for i, (X,y) in enumerate(datasets):
        X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=RANDOM_STATE, test_size=0.33)
        tree = DecisionTreeClassifier(random_state=RANDOM_STATE, **model_kwargs).fit(X_train, y_train)
        plot_surface(tree, X, y)
        if dataset is not None:
            plt.title(dataset_name[dataset])
        else:
            plt.title(dataset_name[i])

        accuracy = lambda pred, true: sum(pred == true) / len(true)

        print(f"Train Acc: {accuracy_score(tree.predict(X_train), y_train)}\nTest Acc: {accuracy_score(tree.predict(X_test), y_test)}")
        # print(f"Train Acc: {accuracy_score(tree.predict(X_train), y_train)}\nTest Acc: {accuracy_score(tree.predict(X_test), y_test)}")
        plt.show()

train_trees(datasets)

__Ответ:__ Да, очень сильно, буквально запомнили обучающую выборку

__Задание 2. (1.5 балла)__

Попробуйте перебрать несколько параметров для регуляризации (напр. `max_depth`, `min_samples_leaf`). Для каждого набора гиперпараметров постройте разделяющую поверхность, выведите обучающую и тестовую ошибки. Можно делать кросс-валидацию или просто разбиение на трейн и тест, главное делайте каждый раз одинаковое разбиение, чтобы можно было корректно сравнивать (помните же, что итоговое дерево сильно зависит от небольшого изменения обучающей выборки?). Проследите как меняется разделяющая поверхность и обобщающая способность. Почему так происходит, одинаково ли изменение для разных датасетов?

In [ ]:
max_depth = [5, 10, 15]
min_samples_leaf = [1, 5, 10]

for d in max_depth:
    for s in min_samples_leaf:
        print("-" * 100, "\n" f"\ndepth: {d}, samples_leaf: {s}\n", "-"*100)
        train_trees(datasets=datasets, dataset=0, max_depth=d, min_samples_leaf=s)


In [ ]:
for d in max_depth:
    for s in min_samples_leaf:
        print("-" * 100, "\n" f"\ndepth: {d}, samples_leaf: {s}\n", "-"*100)
        train_trees(datasets=datasets, dataset=1, max_depth=d, min_samples_leaf=s)


In [ ]:
for d in max_depth:
    for s in min_samples_leaf:
        print("-" * 100, "\n" f"\ndepth: {d}, samples_leaf: {s}\n", "-"*100)
        train_trees(datasets=datasets, dataset=2, max_depth=d, min_samples_leaf=s)


__Ответ:__ Низкий min_sample_leaf приводит к переобучению тк. мы позволяем дереву запомнить каждый объект (распускать корни листья всего для одного объекта). Чем дерево глубже, тем меньше оно генерализирует. Выглядит как будто лучше всего модели обобщаются на датасете classification. Довольно простое дерево на нем может получить хорошие метрики

# 2. Решающие деревья своими руками

В этой части вам нужно реализовать свой класс для обучения решающего дерева в задаче бинарной классификации с возможностью обработки вещественных и категориальных признаков.

__Задание 3. (1.5 балл)__

Реализуйте функцию find_best_split из модуля hw5code.py

Под критерием Джини здесь подразумевается следующая функция:
$$Q(R) = -\frac {|R_l|}{|R|}H(R_l) -\frac {|R_r|}{|R|}H(R_r)$$,
$R$ — множество объектов, $R_l$ и $R_r$ — объекты, попавшие в левое и правое поддерево,
$H(R) = 1-p_1^2-p_0^2$, $p_1$, $p_0$ — доля объектов класса 1 и 0 соответственно.

$H(R) = 1 - p^2_1 - (1-p_1)^2 = 1- p_1^2 - 1 + 2p_1 - p_1^2 =2p_1 - 2p_1^2 \neq 2p_1(1-p_1)$

https://contest.yandex.ru/contest/56884/run-report/102719507/

In [ ]:
# %%time

import numpy as np

def calc_proportion(mask):
    pos_count = np.sum(mask==1, axis=1)
    neg_count = np.sum(mask==0, axis=1)
    pos_prop = pos_count / (pos_count + neg_count)
    neg_prop = 1 - pos_prop

    return pos_prop, neg_prop

def find_best_split(feature_vector: np.array, target_vector: np.array, verbose=False):
    sorted_indeces = np.argsort(feature_vector)
    target_vector = target_vector[sorted_indeces]
    unsorted_features = feature_vector.copy()

    sorted_features = feature_vector[sorted_indeces]
    _, unique_indices = np.unique(sorted_features, return_index=True)
    unique_indices = unique_indices
    unique_features = sorted_features[unique_indices]

    thresholds = (unique_features[1:] + unique_features[:-1]) / 2

    R_left = np.arange(1, target_vector.shape[0])
    R_right = target_vector.shape[0] -R_left

    p1_left = np.cumsum(target_vector[:-1]) / R_left
    p1_right = np.flip( np.cumsum( np.flip (target_vector[1:] ) ) ) / R_right

    l_imp = 1 - np.square(p1_left) - np.square(1 - p1_left)
    r_imp = 1 - np.square(p1_right) - np.square(1 - p1_right)

    ginis = np.sum([-l_imp * R_left / target_vector.shape[0], -r_imp * R_right / target_vector.shape[0]], axis=0)
    if verbose:
        print("unsorted features: ", unsorted_features)
        print("sorted features:", sorted_features)
        print("unique features: ", unique_features)
        print("Thresholds:", thresholds)
        print("ginis:", ginis)
    # takes first entries of sorted_feature_vector and removes first element so shapes cast nicely
    first_entries = unique_indices[1:] - 1
    ginis = ginis[first_entries]
    if verbose:
        print("first_entries: ", first_entries,"\t", sorted_features[first_entries])
        print("ginis on thresholds: ", ginis)

    if np.all(sorted_features == sorted_features[0]):
        best_gini_ind = 0
        thresholds = [0]
        ginis = [0]
    else:
        best_gini_ind = np.argmax(ginis)

    return thresholds, \
            ginis, \
            thresholds[best_gini_ind], \
            ginis[best_gini_ind]

# a, targets = np.array([1,2,2,3,3,3,4]), np.array([0,0,1,0,0,1,1])
# assert a.shape == targets.shape

# find_best_split(a, targets)

In [ ]:
# _, unique_indices = np.unique(np.sort(), return_index=True); unique_indices
# unique_indices[:-1]

__Задание 4. (0.5 балла)__

Загрузите таблицу [students.csv](https://github.com/esokolov/ml-course-hse/blob/master/2022-fall/homeworks-practice/homework-practice-05-trees/students.csv) (это немного преобразованный датасет [User Knowledge](https://archive.ics.uci.edu/ml/datasets/User+Knowledge+Modeling)). В ней признаки объекта записаны в первых пяти столбцах, а в последнем записана целевая переменная (класс: 0 или 1). Постройте на одном изображении пять кривых "порог — значение критерия Джини" для всех пяти признаков. Отдельно визуализируйте scatter-графики "значение признака — класс" для всех пяти признаков.

In [ ]:
df = pd.read_csv("students.csv")

In [ ]:
df.head()

In [ ]:
import seaborn as sns

target = df["UNS"]
names=[]
for name, feature in df.drop("UNS", axis=1).items():
    t, g, _, _ = find_best_split(feature.to_numpy(), target.to_numpy())
    fig = sns.scatterplot(x=t,y=g)
    names += [name]
plt.title("порог — значение критерия Джини")
plt.xlabel("Threshold")
plt.ylabel("Gini")
plt.legend(labels=names)

In [ ]:
target = df["UNS"]
names=[]
for name, feature in df.drop("UNS", axis=1).items():
    fig = sns.scatterplot(x=feature,y=target)
    names += [name]
    plt.title(f"значение признака — класс ({name})")
    plt.xlabel("Value")
    plt.ylabel("Class")
    plt.show()


__Задание 5. (0.5 балла)__

Исходя из кривых значений критерия Джини, по какому признаку нужно производить деление выборки на два поддерева? Согласуется ли этот результат с визуальной оценкой scatter-графиков? Как бы охарактеризовали вид кривой для "хороших" признаков, по которым выборка делится почти идеально? Чем отличаются кривые для признаков, по которым деление практически невозможно?



**Ответ:** PEG. Да, очень заметно как можно понизить impurity правильным порогом. Хорошие признаки похожи на bell-curve, или может на перевернутую параболуб кривые для признаков, по которым деление почти невозможно немного флуктуируют на низких значениях негативного impurity, какой бы порог не поставил, далеко не уйдешь...

__Задание 6. (1.5 балла).__

Разберитесь с уже написанным кодом в классе DecisionTree модуля hw5code.py. Найдите ошибки в реализации метода \_fit_node. Напишите функцию \_predict_node.

 Построение дерева осуществляется согласно базовому жадному алгоритму, предложенному в [лекции](https://github.com/esokolov/ml-course-hse/blob/master/2020-fall/lecture-notes/lecture07-trees.pdf) в разделе «Построение дерева». Выбор лучшего разбиения необходимо производить по критерию Джини. Критерий останова: все объекты в листе относятся к одному классу или ни по одному признаку нельзя разбить выборку. Ответ в листе: наиболее часто встречающийся класс в листе. Для категориальных признаков выполняется преобразование, описанное в лекции в разделе «Учет категориальных признаков».

https://contest.yandex.ru/contest/56884/run-report/102812009/

In [ ]:
import numpy as np
from collections import Counter
from sklearn.base import BaseEstimator, ClassifierMixin


class DecisionTree(BaseEstimator, ClassifierMixin):
    def __init__(self, feature_types, max_depth=None, min_samples_split=None, min_samples_leaf=None, verbose=False, tree={}):
        # if np.any(list(map(lambda x: x != "real" and x != "categorical", feature_types))):
        #     raise ValueError("There is unknown feature type")

        # ["type"] - terminal / nonterminal
        # ["left_child"], ["right_child"]
        # ["class"] - predicted class
        # ["feature_split"] - feature index to split on
        # ["threshold"] - T value for threshold
        # ["categories_split"] - subset of categories which go left

        self.tree = tree
        self.feature_types = feature_types
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.verbose = verbose

    def _fit_node(self, sub_X, sub_y, node):
        if np.all(sub_y == sub_y[0]):  # Use '==' for comparison
            node["type"] = "terminal"
            node["class"] = sub_y[0]
            return

        feature_best, threshold_best, gini_best, split = None, None, None, None
        for feature in range(sub_X.shape[1]):  # Removed  index

            feature_type = self.feature_types[feature]
            categories_map = {}

            if feature_type == "real":
                feature_vector = sub_X[:, feature]
            elif feature_type == "categorical":
                counts = Counter(sub_X[:, feature])
                clicks = Counter(sub_X[sub_y == 1, feature])
                ratio = {}
                for key, current_count in counts.items():
                    if key in clicks:
                        current_click = clicks[key]
                    else:
                        current_click = 0
                    ratio[key] = current_click / current_count  # ratio calculation {"B": 0.3, "A": 0.6, "C": 0.1, }
                sorted_categories = sorted(ratio, key=ratio.get) # ["C", "B", "A"]
                categories_map = dict(zip(sorted_categories, range(len(sorted_categories)))) # {"C": 0, "B": 1, "A": 2}

                feature_vector = np.array(list(map(lambda x: categories_map[x], sub_X[:, feature]))) # ["C", "B"] -> [0, 1]
            else:
                raise ValueError("Invalid feature type")

            if len(set(feature_vector)) == 1:  # uniqueness
                continue

            _, _, threshold, gini = find_best_split(feature_vector, sub_y, verbose=self.verbose)
            if gini_best is None or gini > gini_best:
                feature_best = feature
                gini_best = gini
                split = feature_vector < threshold

                if feature_type == "real":
                    threshold_best = threshold
                elif feature_type == "categorical":
                    # such categories that go into left sub-tree
                    threshold_best = [x[0] for x in categories_map.items() if x[1] < threshold]

                else:
                    raise ValueError("Invalid feature type")

        if feature_best is None:
            node["type"] = "terminal"
            node["class"] = Counter(sub_y).most_common(1)[0][0]  # Extract the class from the most_common result
            return

        node["type"] = "nonterminal"

        node["feature_split"] = feature_best
        if self.feature_types[feature_best] == "real":
            node["threshold"] = threshold_best
        elif self.feature_types[feature_best] == "categorical":
            node["categories_split"] = threshold_best
        else:
            raise ValueError("Invalid feature type")

        node["left_child"], node["right_child"] = {}, {}
        self._fit_node(sub_X[split], sub_y[split], node["left_child"])
        self._fit_node(sub_X[np.logical_not(split)], sub_y[np.logical_not(split)], node["right_child"])



    def _predict_node(self, x: np.array, node):
        # x - one observation (vector)
        if node["type"] == "terminal":
            return node["class"]
        elif node["type"] == "nonterminal":
            split_feature_i = node["feature_split"]
            feature_type = self.feature_types[split_feature_i]
            # print(f"x: {x}, split_feature_i: {split_feature_i}")
            splitting_feature = x[split_feature_i]

            if feature_type == "real":
                if splitting_feature < node["threshold"]:
                    return self._predict_node(x, node["left_child"])
                else:
                    return self._predict_node(x, node["right_child"])
            elif feature_type == "categorical":
                if splitting_feature in node["categories_split"]:
                    return self._predict_node(x, node["left_child"])
                else:
                    return self._predict_node(x, node["right_child"])
            else:
                raise ValueError
            self.log_node(node)
        else:
            raise ValueError

    def fit(self, X, y):
        assert(len(y) > 0), len(y)

        self._fit_node(X, y, self.tree)
        return self

    def predict(self, X):
        predicted = []
        for x in X:
            #x is row (one observation)
            predicted.append(self._predict_node(x, self.tree))
        return np.array(predicted)

    def log_node(self,tree):
        split_feature_i = tree["feature_split"]
        feature_type = self.feature_types[split_feature_i]
        info = f"""
["type"] = {tree.get("type")}
["class"] = {tree.get("class")}
["feature_split"] = {tree.get("feature_split")}
["threshold"] = {tree.get("threshold")}
["categories_split"] = {tree.get("categories_split")}
feature_type = {feature_type}
        """
        print(info)

tree = DecisionTree(feature_types, verbose=False)

X_cut = X_train.to_numpy()[:20,:]
y_cut = y_train.to_numpy()[:20]
tree.fit(X_cut, y_cut)

DecisionTree(feature_types=['categorical', 'categorical', 'categorical',
                            'categorical', 'categorical', 'categorical',
                            'categorical', 'categorical', 'categorical',
                            'categorical', 'categorical', 'categorical',
                            'categorical', 'categorical', 'categorical',
                            'categorical', 'categorical', 'categorical',
                            'categorical', 'categorical', 'categorical',
                            'categorical'])

__Задание 7. (0.5 балла)__

Протестируйте свое решающее дерево на датасете [mushrooms](https://archive.ics.uci.edu/ml/datasets/Mushroom). Вам нужно скачать таблицу agaricus-lepiota.data (из [Data Folder](https://archive.ics.uci.edu/ml/machine-learning-databases/mushroom/)), прочитать ее с помощью pandas, применить к каждому столбцу LabelEncoder (из sklearn), чтобы преобразовать строковые имена категорий в натуральные числа. Первый столбец — это целевая переменная (e — edible, p — poisonous) Мы будем измерять качество с помощью accuracy, так что нам не очень важно, что будет классом 1, а что — классом 0. Обучите решающее дерево на половине случайно выбранных объектов (признаки в датасете категориальные) и сделайте предсказания для оставшейся половины. Вычислите accuracy.

У вас должно получиться значение accuracy, равное единице (или очень близкое к единице), и не очень глубокое дерево.

In [ ]:
df = pd.read_csv("agaricus-lepiota.data", sep=",")

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

df = df.apply(LabelEncoder().fit_transform); df.head()

target = df["p"]
X = df.drop("p", axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, target, test_size=0.5)

In [ ]:
feature_types = ["categorical"] * X.shape[1]

tree = DecisionTree(feature_types)

In [ ]:
tree = DecisionTree(feature_types)
tree.fit(X_train.to_numpy(), y_train.to_numpy())

In [ ]:
preds = tree.predict(X_test.to_numpy())

In [ ]:
from sklearn.metrics import accuracy_score

print(accuracy_score(y_test.to_numpy(), preds))

1.0


In [ ]:
tree._tree # НЕ ГЛУБОКОЕ

{'type': 'nonterminal',
 'feature_split': 4,
 'categories_split': [0, 3, 5],
 'left_child': {'type': 'nonterminal',
  'feature_split': 19,
  'categories_split': [2, 3, 4, 6, 8, 0, 1, 7],
  'left_child': {'type': 'nonterminal',
   'feature_split': 14,
   'categories_split': [6, 3, 5, 2, 7, 4],
   'left_child': {'type': 'nonterminal',
    'feature_split': 1,
    'categories_split': [2, 0, 3],
    'left_child': {'type': 'nonterminal',
     'feature_split': 14,
     'categories_split': [6, 3, 5, 2, 7],
     'left_child': {'type': 'nonterminal',
      'feature_split': 0,
      'categories_split': [5, 4, 2, 0, 3],
      'left_child': {'type': 'nonterminal',
       'feature_split': 21,
       'categories_split': [1, 0, 3, 4, 6, 5],
       'left_child': {'type': 'terminal', 'class': 0},
       'right_child': {'type': 'nonterminal',
        'feature_split': 1,
        'categories_split': [2],
        'left_child': {'type': 'terminal', 'class': 0},
        'right_child': {'type': 'terminal', 'cl

__Задание 8. (бонус, 1 балл)__

Реализуйте в классе DecisionTree поддержку параметров max_depth, min_samples_split и min_samples_leaf по аналогии с DecisionTreeClassifier. Постройте графики зависимости качества предсказания в зависимости от этих параметров для набора данных tic-tac-toe (см. следующий пункт).

__Задание 9. (2 балла)__

Загрузите следующие наборы данных (напомним, что pandas умеет загружать файлы по url, в нашем случае это файл \*.data), предварительно ознакомившись с описанием признаков и целевой переменной в каждом из них (она записаны в Data Folder, в файле *.names):
* [mushrooms](https://archive.ics.uci.edu/ml/datasets/Mushroom) (загрузили в предыдущем пункте, классы записаны в нулевом столбце),
* [tic-tac-toe](https://archive.ics.uci.edu/ml/datasets/Tic-Tac-Toe+Endgame) (классы записаны в последнем столбце)
* [cars](https://archive.ics.uci.edu/ml/datasets/Car+Evaluation) (классы записаны в последнем столбце, считаем что unacc, acc — это класс 0, good, vgood — класс 1)
* [nursery](https://archive.ics.uci.edu/ml/datasets/Nursery) (классы записаны в последнем столбце, считаем, что not_recom и recommend — класс 0, very_recom, priority, spec_prior — класс 1).

Закодируйте категориальные признаки, использовав LabelEncoder. С помощью cross_val_score (cv=10) оцените accuracy на каждом из этих наборов данных следующих алгоритмов:
* DecisionTree, считающий все признаки вещественными
* DecisionTree, считающий все признаки категориальными
* DecisionTree, считающий все признаки вещественными + one-hot-encoding всех признаков
* DecisionTreeClassifier из sklearn. Запишите результат в pd.DataFrame (по строкам — наборы данных, по столбцам — алгоритмы).

Рекомендации:
* Чтобы cross_val_score вычисляла точность, нужно передать scoring=make_scorer(accuracy_score), обе фукнции из sklearn.metrics.
* Если вам позволяет память (а она скорее всего позволяет), указывайте параметр sparse=False в OneHotEncoder (если вы, конечно, используете его). Иначе вам придется добиваться того, чтобы ваша реализация дерева умела работать с разреженными матрицами (что тоже, в целом, не очень сложно).

In [ ]:
!pip install ucimlrepo -q

In [ ]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
mushroom = fetch_ucirepo(id=73)
# data (as pandas dataframes)
X_mushroom = mushroom.data.features
y_mushroom = mushroom.data.targets

y_mushroom[y_mushroom.isin(["p"])] = 1
y_mushroom[~y_mushroom.isin(["p"])] = 0


tictac = pd.read_csv("tic-tac-toe.data", sep=",")
target_col = "positive"
X_tictac = tictac.drop(target_col, axis=1)
y_tictac = tictac[target_col]

y_tictac[y_tictac.isin(["positive"])] = 1
y_tictac[~y_tictac.isin(["positive"])] = 0

# fetch dataset
car_evaluation = fetch_ucirepo(id=19)
# data (as pandas dataframes)
X_car = car_evaluation.data.features
y_car = car_evaluation.data.targets

y_car[y_car.isin(["good", "vgood"])] = 1
y_car[~y_car.isin(["good", "vgood"])] = 0

nursery = pd.read_csv("nursery.data", sep=",")
target_col = "recommend"
X_nursery = nursery.drop(target_col, axis=1)
y_nursery = nursery[target_col]

y_nursery[y_nursery.isin(["very_recom", "priority", "spec_prior"])] = 1
y_nursery[~y_nursery.isin(["very_recom", "priority", "spec_prior"])] = 0

Xs = [X_mushroom, X_tictac, X_car, X_nursery]
ys = [y_mushroom, y_tictac, y_car, y_nursery]

In [ ]:
Xs = [x.apply(LabelEncoder().fit_transform) for x in Xs]

In [ ]:
ys = [np.nan_to_num(y) for y in ys]

for y in ys:
    print(y[~np.isin(y, [0,1])])

[]
[]
[]
[]


In [ ]:
from sklearn.utils.estimator_checks import check_estimator

check_estimator(DecisionTree(["real"] * 10))

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier as DTC

def build_tree_real(X):
    feature_types = ["real"] * X.shape[1]
    return DecisionTree(feature_types), None

def build_tree_cat(X):
    feature_types = ["categorical"] * X.shape[1]
    return DecisionTree(feature_types), None

def build_tree_ohe(X):
    feature_types = ["real"] * X.shape[1]
    enc = OneHotEncoder(handle_unknown='ignore')
    return DecisionTree(feature_types), enc.fit_transform(X)

def build_tree_og(X):
    return DTC(), None

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer

functions = [build_tree_real,build_tree_cat,build_tree_ohe,build_tree_og]
datasets = ["mushroom","tictac","car_evaluation","nursery"]
results = pd.DataFrame(columns = [alg.__name__ for alg in functions])
CV = 10

for (X, y, dataset) in zip(Xs, ys, datasets):
    X = X.to_numpy()
    if not isinstance(y, np.ndarray):
        y = y.to_numpy()
    X = X.astype("int")
    y = y.astype("int")

    algos = {}
    for algorithm in functions:
        tree, X_trans = algorithm(X)
        if X_trans is not None:
            X = X_trans
        print(X.shape, y.shape)
        scores = cross_val_score(tree, X, y, cv=CV, scoring=make_scorer(accuracy_score))
        algos[algorithm.__name__] = [np.mean(scores)]
    # results.iloc[dataset] = algos
    df = pd.DataFrame(algos)
    results = pd.concat([results, df], ignore_index=True)

results.index = datasets
print("Accuracy: ")
results

(8124, 22) (8124, 1)
(8124, 22) (8124, 1)
(8124, 117) (8124, 1)
(8124, 117) (8124, 1)
(957, 9) (957,)
(957, 9) (957,)
(957, 27) (957,)
(957, 27) (957,)
(1728, 6) (1728, 1)
(1728, 6) (1728, 1)
(1728, 21) (1728, 1)
(1728, 21) (1728, 1)
(12959, 8) (12959,)
(12959, 8) (12959,)
(12959, 27) (12959,)
(12959, 27) (12959,)
Accuracy: 


,build_tree_real,build_tree_cat,build_tree_ohe,build_tree_og
mushroom,1.0,1.0,1.0,1.0
tictac,1.0,1.0,1.0,1.0
car_evaluation,1.0,1.0,1.0,1.0
nursery,1.0,1.0,1.0,1.0


__Задание 10. (1 балла)__

Проанализируйте результаты эксперимента.
Одинаково ли для разных наборов данных ранжируются алгоритмы?
Порассуждайте, почему так происходит.

Обратите внимание на значение признаков в разных наборах данных.
Присутствует ли в результатах какая-то компонента случайности?
Можно ли повлиять на нее и улушить работу алгоритмов?

**Ответ:**

Вставьте что угодно, описывающее ваши впечатления от этого задания: